In [1]:
#Setup

first_year = 0 #earliest year to include in data wrangling
#note: after running averages this will mean the earliest year in the model will be later than this
exclude_finals = 1 #1 = exclude, 2 = include
games_to_keep = 'mix' #'mix' = mix of home and away, 'home' 'away' = home and away respectively. Else

In [2]:
import pandas as pd
import numpy as np

In [3]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)
def newMeanCol(df, col_nm, N):
    new_col = col_nm + '_mean' + str(N)
    x = df.loc[0:,col_nm]
    mean_vals = list(np.insert(
        running_mean(
            np.array(x), N), 0, [np.nan] * N))
    df[new_col] = mean_vals[0:len(x)]
    return df

In [4]:
def colLookup(data, string):
    return data[[col for col in data.columns if string  in col]]

In [5]:
#Where a row is na (i.e. it is in the first x games for that team and therefore has no mean), remove both rows of that game.
#This is for the instance when new teams join the league, can't calculate the score for the older team so have to remove both instances.
def NANConsolidate(df, col):
    na_game_ids = set(df[(df.isna().any(axis = 1))][col])
    return df[~df[col].isin(na_game_ids)]

In [6]:
def CheckTeam(df, team):
    df = df[df['team'] == team]
    min_game = min(df.date)
    max_game = max(df.date)
    n_na = sum(df.isna().any(axis=1))
    return print(min_game, max_game, n_na)

In [7]:
###Initial load and clean data
afl_DF_main = pd.read_pickle('afl_DF.pkl') #Read in the data
afl_DF_main = afl_DF_main[afl_DF_main.year >= first_year].reset_index() #Filter the data toonly include data from years greater than or equal to <first_year>
afl_DF = afl_DF_main.loc[0:,~(afl_DF_main.isna().any())] #Remove any columns that have NAs in the columns
afl_DF.columns.duplicated()
if exclude_finals == 1: #Drop all finals games when <exclude_finals> is 1
    afl_DF = afl_DF[afl_DF_main['final'] == 0]

###Create an index for each of the rounds
round_index = afl_DF.loc[0:][['year','season_round']] #New dataset with year and season_round
round_index = round_index.drop_duplicates(subset = ['year', 'season_round']) #Remove duplicate round rows
round_index = round_index.sort_values(by = ['year', 'season_round'], #Order the rounds by year and round, reset the index
                       ascending = [True, True]).reset_index(drop=True) 
round_index['round_index'] = pd.DataFrame(list(round_index.index)) #Create a new column with the index
afl_DF = pd.merge(afl_DF, #Join the index back to the main data
        round_index,
        left_on = ['year','season_round'],
        right_on = ['year','season_round'],
        how = 'left')

###Create an index for each of the games
game_index = afl_DF.loc[0:][['fw_game_id','date']] #New dataset with fw_game_id and date
game_index = game_index.drop_duplicates(subset = ['fw_game_id','date']) #Remove duplicate game rows
game_index = game_index.sort_values(by = ['date', 'fw_game_id'], #Order the games by date and fw_game_id, reset the index
                       ascending = [True, True]
                                   ).reset_index(drop=True)
game_index['game_index'] = pd.DataFrame(list(game_index.index)) #Create a new column with the index
afl_DF = pd.merge(afl_DF, #Join the index back to the main data
        game_index,
        left_on = ['fw_game_id','date'],
        right_on = ['fw_game_id','date'],
        how = 'left')

In [8]:
###Sort data by team and game_index for running mean
afl_DF = afl_DF.sort_values(by = ['team','game_index'],
             ascending = [True, True]
                 ).reset_index(drop=True)

In [9]:
###Create new score, margin, result and win variables
afl_DF['ScoreTeam'] = (afl_DF['Goals_KickedTeam'] * 6) + afl_DF['Behinds_KickedTeam'] #Goals for the team * 6 plus behinds for the team is the score 
afl_DF['ScoreOppnt'] = (afl_DF['Goals_KickedOppnt'] * 6) + afl_DF['Behinds_KickedOppnt'] #Goals for the opponent * 6 plus behinds for the opponent is the score 
afl_DF['Margin'] = afl_DF['ScoreTeam'] - afl_DF['ScoreOppnt'] #The final margin calculated as the score for the team minus the score for the opponent 

In [10]:
afl_DF.loc[afl_DF['Margin'] == 0, 'Result'] = 0 #When margin is 0, 0
afl_DF.loc[afl_DF['Margin'] > 0, 'Result'] = 1 #When margin > 0, 1
afl_DF.loc[afl_DF['Margin'] < 0, 'Result'] = -1 #When margin < 0, -1

afl_DF.loc[:, 'Win'] = 0 #Make win a zero
afl_DF.loc[afl_DF['Margin'] > 0, 'Win'] = 1 #When margin > 0 it's a win = 1

In [11]:
###Statistics to remove from the data
drop_stat_cols = ['Kicks', 'Handballs','Marks', 'Tackles', 'Hitouts', 'Frees_For', 'Frees_Against',
       'Goals_Kicked', 'Behinds_Kicked', 'Rushed_Behinds', 'Disposals_Per_Goal', 'Clearances',
       'Clangers', 'In50s_Per_Scoring_Shot', 'Inside_50s_Per_Goal',  'In50s_Goal',
       'Contested_Possessions', 'Uncontested_Possessions', 'Effective_Disposals', 'Contested_Marks',
        'One_Percenters', 'Bounces', 'Turnovers', 'Intercepts', 'Tackles_Inside_50', 'Kick_to_Handball_Ratio']
drop_stat_cols_fin = [string + 'Team' for string in drop_stat_cols]
drop_stat_cols_fin.extend([string + 'Oppnt' for string in drop_stat_cols_fin])
drop_stat_cols_fin = [col for col in drop_stat_cols_fin if col in afl_DF.columns]

In [12]:
#Need to check if this is doing anything
afl_DF = afl_DF.loc[:,~afl_DF.columns.duplicated()]

In [13]:
####Create a column representing the % of each 'stat' achieved by "team"

team_array = colLookup(afl_DF, 'Team').values #get all columns with "Team" in the name, then return the cell values
oppnt_array = colLookup(afl_DF, 'Oppnt').values #get all columns with "Opponent" in the name, then return the cell values
diff_array = team_array / (team_array + oppnt_array + 0.00001) #Create an array of values which is the % of the stat belonging to team. Note: adding small positive value to avoid inf
afl_DF[list(colLookup(afl_DF, 'Team').columns.str.replace('Team','Pcnt'))] = pd.DataFrame(diff_array) #Create a new set of columns and insert the array of values back into main dataframe

In [14]:
afl_DF = afl_DF[[col for col in afl_DF.columns if 'Team' not in col and 'Oppnt' not in col]]
#afl_DF = afl_DF[[col for col in afl_DF.columns if 'Oppnt' not in col]]
#afl_DF = afl_DF[[col for col in afl_DF.columns if 'Team' not in col]]

In [15]:
ELO_col = ['Margin']
ELO_mean_N = [8]
for n in ELO_mean_N:
    for col in ELO_col:
        afl_DF = afl_DF.groupby('team').apply(newMeanCol, col_nm = col, N = n)
afl_DF = NANConsolidate(afl_DF, 'fw_game_id')

In [16]:
afl_DF.columns = afl_DF.columns.str.replace("_mean","_T_ELO_mean")

In [17]:
cols = ['fw_game_id', 'team']
cols.extend([col for col in afl_DF.columns if 'ELO' in col])
elo_DF = afl_DF[cols]
elo_DF.columns = elo_DF.columns.str.replace('T_ELO','O_ELO')
elo_DF.columns = elo_DF.columns.str.replace('team','opponent')

In [18]:
afl_DF = pd.merge(afl_DF,
         elo_DF,
         left_on = ['fw_game_id','opponent'],
         right_on = ['fw_game_id','opponent'],
        how = 'left')
ELO_mean_N = [8]
afl_DF = afl_DF[(afl_DF["round_index"] > (max(ELO_mean_N) + 2))]
afl_DF['Margin_ELO_m8'] = pd.DataFrame(afl_DF.Margin - (afl_DF.Margin_T_ELO_mean8 - afl_DF.Margin_O_ELO_mean8))
afl_DF = afl_DF.drop(['Margin_T_ELO_mean8','Margin_O_ELO_mean8'], axis = 1)

In [19]:
cols_to_apply = [col for col in afl_DF.columns if 'Pcnt' in col]
cols_to_apply.extend([col for col in afl_DF.columns if 'ELO' in col])
cols_to_apply.extend(['Win','Margin'])

In [20]:
mean_N = [8, 20]
for N in mean_N:
    for col in cols_to_apply:
        afl_DF = afl_DF.groupby('team').apply(newMeanCol, col_nm = col, N = N)
afl_DF = NANConsolidate(afl_DF, 'fw_game_id')

In [21]:
afl_DF = afl_DF[~(afl_DF.isna().any(axis = 1))]

In [22]:
afl_DF = afl_DF[[col for col in afl_DF.columns if 'Pcnt' not in col or 'mean' in col]]

In [23]:
feature_col = ['fw_game_id','team']
feature_col.extend([col for col in afl_DF.columns if 'mean' in col])
afl_DF_oppnt = afl_DF[feature_col]
afl_DF_oppnt.columns = afl_DF_oppnt.columns.str.replace('team','opponent')
afl_DF_oppnt.columns = afl_DF_oppnt.columns.str.replace('mean','O_mean')
afl_DF.columns = afl_DF.columns.str.replace('mean','T_mean')

In [24]:
team_pcnt_col = [col for col in afl_DF.columns if 'mean' in col]
oppnt_pcnt_col = [col for col in afl_DF_oppnt.columns if 'mean' in col]
team_pcnt_col = [col for col in team_pcnt_col if 'Win_' not in col and 'Margin_' not in col]
oppnt_pcnt_col = [col for col in oppnt_pcnt_col if 'Win_' not in col and 'Margin_' not in col]

team_other_col = [col for col in afl_DF.columns if ('Win_' in col or 'Margin_' in col) and '_T_' in col]
oppnt_other_col = [col for col in afl_DF_oppnt.columns if ('Win_' in col or 'Margin_' in col) and '_O_' in col]

In [25]:
afl_DF = pd.merge(afl_DF,
                 afl_DF_oppnt,
                 left_on = ['fw_game_id','opponent'],
                 right_on = ['fw_game_id','opponent'],
                 how='left')

In [26]:
team_pcnt_mean = afl_DF[team_pcnt_col].values
oppnt_pcnt_mean = afl_DF[oppnt_pcnt_col].values
diff_pcnt_mean = team_pcnt_mean / (team_pcnt_mean + oppnt_pcnt_mean + 0.000001)

team_other_mean = afl_DF[team_other_col].values
oppnt_other_mean = afl_DF[oppnt_other_col].values
diff_other_mean = team_other_mean - oppnt_other_mean

In [27]:
afl_DF[[col.replace('T','D') for col in team_pcnt_col]] = pd.DataFrame(diff_pcnt_mean)
afl_DF[[col.replace('T','D') for col in team_other_col]] = pd.DataFrame(diff_other_mean)

In [28]:
afl_DF = afl_DF[[col for col in afl_DF.columns if '_T_' not in col and '_O_' not in col]]

In [29]:
afl_DF = afl_DF[~(afl_DF.Result == 0)]

In [30]:
drop_cols = ['season_round','round_char','date','team','opponent', 'location','attendance','final',
             'team_odds','oppnt_odds','team_line','oppnt_line', 'Margin','Result','Margin_ELO_m8','game_index']
drop_cols_fin = [col for col in drop_cols if col in afl_DF.columns]
afl_DF = afl_DF.drop(drop_cols, axis = 1)

In [31]:
cols = list(afl_DF.columns)
cols.remove('Win')
cols.append('Win')
afl_DF = afl_DF[cols]

In [33]:
if games_to_keep not in ['mix','home','away','all']:
    print('error, <games_to_keep> variable not set correctly')
elif games_to_keep == 'mix':
    evens = [x for x in afl_DF.fw_game_id if int(x) % 2 == 0]
    odds = [x for x in afl_DF.fw_game_id if int(x) % 2 > 0]
    a = afl_DF[afl_DF.fw_game_id.isin(odds) & afl_DF.home_game == 1]
    b = afl_DF[afl_DF.fw_game_id.isin(odds) & afl_DF.home_game == 0]
    afl_DF = pd.concat([a,b])
    del a, b
elif games_to_keep == 'home':
    afl_DF = afl_DF[afl_DF.home_game == 1]
elif games_to_keep == 'away':
    afl_DF = afl_DF[afl_DF.home_game == 0]

In [36]:
afl_ML.to_pickle('afl_ML.pkl')